In [1]:
# from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [2]:
# 배운 앙상블 모델을 각각 적용해서
# 리포트 만들고
# 최상의 성능을 내는 모델을 가지고
# 정식으로 학습해서
# 예측

In [3]:
cols = ['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
#  'MEDV'
 ]
df = pd.DataFrame(data,columns=cols)
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [4]:
# 모델생성
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor,AdaBoostRegressor

In [5]:
models = [RandomForestRegressor(), ExtraTreesRegressor(),GradientBoostingRegressor(),
  HistGradientBoostingRegressor(),AdaBoostRegressor()
]

In [6]:
scores = []
for model in models:
  s = cross_validate(model,data,target)
  scores.append(s['test_score'].mean())

In [7]:
import numpy as np
max_index = np.argmax(scores)
scores, max_index

([0.6241546645026377,
  0.6458261438792163,
  0.6754260682593529,
  0.5491141304411663,
  0.6092847524210161],
 2)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data,target,random_state=42)
best_model = GradientBoostingRegressor()

In [9]:
best_model.fit(x_train,y_train)

GradientBoostingRegressor()

In [10]:
# 평가
best_model.score(x_train,y_train), best_model.score(x_test,y_test)

(0.9785520655757387, 0.8761136572687576)

In [ ]:
# 하이퍼 파라메터 튜닝
param = {
    'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate' : [1,0.1,0.01,0.001],
    'n_estimators' : [10,100,200,500],
    'subsample':[1.0,2.0,3.0]
}
from sklearn.model_selection import GridSearchCV
gsc = GridSearchCV(
    GradientBoostingRegressor(),
    param_grid = param,
    n_jobs=-1
)
gsc.fit(x_train,y_train)

In [12]:
gsc.best_params_

{'learning_rate': 0.1, 'loss': 'huber', 'n_estimators': 200, 'subsample': 1.0}

In [13]:
best_model = gsc.best_estimator_

In [14]:
best_model.score(x_train,y_train), best_model.score(x_test,y_test)

(0.989203745748212, 0.8647953594113333)

In [23]:
best_model2 = GradientBoostingRegressor(learning_rate= 0.1, loss= 'huber',
                                        n_estimators= 50, subsample= 1.0,min_samples_split=10)
best_model2.fit(x_train,y_train)
best_model2.score(x_train,y_train), best_model.score(x_test,y_test)

(0.9544568686444863, 0.8647953594113333)